In [20]:
import pandas as pd
import numpy as np
import plotly.express as px

from pprint import pprint as pp

In [29]:
df1 = pd.read_csv("data\\Eglin_Analysis\\Eglin - Sept CBX 1.csv")
df2 = pd.read_csv("data\\Eglin_Analysis\\Eglin - Sept CBX 2 .csv")
df3 = pd.read_csv("data\\Eglin_Analysis\\Eglin - Sept MTR_POA.csv")

df_index = df1['Date (-05:00)']


In [47]:
all_df = pd.concat([df1.reset_index(drop=True),
                    df2.reset_index(drop=True), 
                    df3.reset_index(drop=True)], axis=1,ignore_index=True)

df_columns = [list(df1.columns),
              list(df2.columns),
              list(df3.columns),]

flatten = lambda nested_list: [item for sublist in nested_list for item in sublist]
all_df.columns = flatten(df_columns)

for i in all_df.columns:
    if "Date (-05:00)" in i:
        new_df = all_df.drop([i], axis=1)


new_df = new_df.set_index([df_index])
new_df.index = pd.to_datetime(new_df.index)
new_df = new_df[~new_df.index.duplicated(keep='first')]
new_df

,Recombiner 101 - Current_String_1 (A),Recombiner 101 - Current_String_2 (A),Recombiner 101 - Current_String_3 (A),Recombiner 101 - Current_String_4 (A),Recombiner 101 - Current_String_5 (A),Recombiner 101 - Current_String_6 (A),Recombiner 101 - Current_String_7 (A),Recombiner 101 - Current_String_8 (A),Recombiner 101 - Current_String_9 (A),Recombiner 101 - Current_String_10 (A),...,Recombiner 207 - Current_String_12 (A),Recombiner 207 - Current_String_13 (A),Recombiner 207 - Current_String_14 (A),Recombiner 207 - Current_String_15 (A),Recombiner 207 - Current_String_16 (A),Recombiner 207 - Current_String_17 (A),Recombiner 207 - Current_String_18 (A),Recombiner 207 - Current_String_19 (A),Meter AC Power (W),Sensors - Irradiance reference (W/m²)
Date (-05:00),,,,,,,,,,,,,,,,,,,,,
2022-09-01 00:05:00,0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3,0.2,-0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2022-09-01 00:10:00,0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3,0.2,-0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2022-09-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3,0.2,-0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2022-09-01 00:20:00,0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3,0.2,-0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2022-09-01 00:25:00,0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3,0.3,-0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30 23:40:00,0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3,0.2,-0.2,...,-0.1,0.0,-0.2,0.1,0.3,0.2,0.2,0.0,0,NaN
2022-09-30 23:45:00,0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3,0.2,-0.2,...,-0.1,0.0,-0.2,0.2,0.3,0.2,0.2,0.0,0,NaN
2022-09-30 23:50:00,0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3,0.3,-0.2,...,-0.1,0.0,-0.2,0.2,0.3,0.2,0.2,0.0,0,NaN


In [49]:

curtailment_limit = 30000000

new_df = new_df.between_time("7:00", "17:00")
new_df = new_df[(new_df.loc[:,"Sensors - Irradiance reference (W/m²)"] > 100)]
new_df = new_df[(new_df.loc[:,"Meter AC Power (W)"]<curtailment_limit*0.99)&(new_df.loc[:,"Meter AC Power (W)"] > curtailment_limit*0.01)]
new_df = new_df.fillna(0)

new_df

,Recombiner 101 - Current_String_1 (A),Recombiner 101 - Current_String_2 (A),Recombiner 101 - Current_String_3 (A),Recombiner 101 - Current_String_4 (A),Recombiner 101 - Current_String_5 (A),Recombiner 101 - Current_String_6 (A),Recombiner 101 - Current_String_7 (A),Recombiner 101 - Current_String_8 (A),Recombiner 101 - Current_String_9 (A),Recombiner 101 - Current_String_10 (A),...,Recombiner 207 - Current_String_12 (A),Recombiner 207 - Current_String_13 (A),Recombiner 207 - Current_String_14 (A),Recombiner 207 - Current_String_15 (A),Recombiner 207 - Current_String_16 (A),Recombiner 207 - Current_String_17 (A),Recombiner 207 - Current_String_18 (A),Recombiner 207 - Current_String_19 (A),Meter AC Power (W),Sensors - Irradiance reference (W/m²)
Date (-05:00),,,,,,,,,,,,,,,,,,,,,
2022-09-01 07:50:00,11.0,10.5,10.7,11.7,11.8,12.5,10.5,12.3,9.8,12.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1178290,108.5620
2022-09-01 07:55:00,12.7,12.1,12.5,13.7,13.7,14.5,12.1,14.2,11.4,14.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1404060,124.8220
2022-09-01 08:00:00,14.7,14.0,14.3,15.7,15.7,16.5,13.9,16.2,12.9,16.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1627140,134.1460
2022-09-01 08:05:00,17.2,16.3,16.7,18.3,18.4,19.1,16.1,18.7,14.9,19.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1915210,153.9910
2022-09-01 08:10:00,19.6,18.5,19.0,20.9,20.9,21.7,18.2,21.1,16.8,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2213880,171.2070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30 16:40:00,108.1,111.3,109.2,109.7,104.9,98.7,79.7,90.4,60.6,50.4,...,36.2,40.9,49.1,61.8,62.4,44.1,44.0,40.5,11337900,352.1967
2022-09-30 16:45:00,100.0,101.5,96.1,88.7,71.4,44.9,79.8,97.5,75.5,69.2,...,21.8,23.2,22.7,22.9,22.9,17.7,17.8,16.3,7082400,311.2500
2022-09-30 16:50:00,80.8,79.0,58.8,36.4,22.7,20.9,50.8,37.0,18.2,17.8,...,81.7,87.6,86.7,77.2,71.4,57.4,59.7,46.4,7457900,225.3713


In [50]:
plant = "Gulf I"

day_df = new_df.resample("1D").sum()


dc_meta = pd.read_excel("Project_Metadata_tables.xlsx", sheet_name="Combiner_table").filter(items= ['plant_name', 'string_count','DC Rating Power kW', 'Module IMP'], axis=1)
day_df = day_df.filter(regex="Current")

dc_issues = dc_meta[dc_meta["plant_name"]==plant]  
dc_rating = [i for i in dc_issues['DC Rating Power kW'].values]

relative_dc = []

max_dc = max(dc_rating)

for i in dc_rating:
    dc = (i/max_dc)
    relative_dc.append(dc)

string_count = [s for s in dc_issues['string_count']]

cmb_relative = day_df.div(relative_dc)

cmb_relative.loc[:,'norm'] = cmb_relative.apply(pd.Series.nlargest, axis = 1, n = 5).median(axis = 1)
t = cmb_relative.div(cmb_relative['norm'], axis = 0).clip(lower = 0, upper = 1)
t = t.iloc[:,:-1]
t = t.fillna(0)

down_strings = (string_count*(1-t)).round() 
down_strings


,Recombiner 101 - Current_String_1 (A),Recombiner 101 - Current_String_2 (A),Recombiner 101 - Current_String_3 (A),Recombiner 101 - Current_String_4 (A),Recombiner 101 - Current_String_5 (A),Recombiner 101 - Current_String_6 (A),Recombiner 101 - Current_String_7 (A),Recombiner 101 - Current_String_8 (A),Recombiner 101 - Current_String_9 (A),Recombiner 101 - Current_String_10 (A),...,Recombiner 207 - Current_String_10 (A),Recombiner 207 - Current_String_11 (A),Recombiner 207 - Current_String_12 (A),Recombiner 207 - Current_String_13 (A),Recombiner 207 - Current_String_14 (A),Recombiner 207 - Current_String_15 (A),Recombiner 207 - Current_String_16 (A),Recombiner 207 - Current_String_17 (A),Recombiner 207 - Current_String_18 (A),Recombiner 207 - Current_String_19 (A)
Date (-05:00),,,,,,,,,,,,,,,,,,,,,
2022-09-01,2.0,10.0,9.0,0.0,0.0,0.0,0.0,10.0,10.0,8.0,...,88.0,70.0,88.0,87.0,88.0,88.0,90.0,70.0,70.0,71.0
2022-09-02,10.0,16.0,13.0,4.0,4.0,2.0,2.0,12.0,11.0,11.0,...,6.0,5.0,10.0,4.0,5.0,6.0,26.0,8.0,7.0,10.0
2022-09-03,6.0,15.0,14.0,6.0,7.0,6.0,3.0,15.0,14.0,13.0,...,5.0,6.0,10.0,4.0,6.0,6.0,24.0,6.0,5.0,9.0
2022-09-04,9.0,16.0,16.0,7.0,6.0,6.0,4.0,15.0,14.0,14.0,...,5.0,6.0,10.0,4.0,5.0,4.0,22.0,3.0,0.0,2.0
2022-09-05,8.0,15.0,15.0,7.0,8.0,6.0,3.0,13.0,13.0,13.0,...,5.0,6.0,10.0,4.0,5.0,6.0,24.0,6.0,5.0,8.0
2022-09-06,8.0,16.0,14.0,5.0,5.0,4.0,3.0,13.0,12.0,10.0,...,5.0,12.0,15.0,7.0,6.0,6.0,24.0,6.0,6.0,11.0
2022-09-07,4.0,12.0,12.0,3.0,2.0,1.0,1.0,12.0,11.0,10.0,...,4.0,4.0,8.0,2.0,3.0,3.0,22.0,3.0,2.0,6.0
2022-09-08,5.0,14.0,13.0,6.0,6.0,5.0,2.0,13.0,14.0,14.0,...,6.0,7.0,11.0,5.0,6.0,6.0,24.0,6.0,4.0,6.0
2022-09-09,12.0,18.0,14.0,3.0,2.0,1.0,6.0,16.0,13.0,12.0,...,4.0,3.0,7.0,1.0,2.0,5.0,24.0,6.0,3.0,5.0


In [43]:
down_strings.to_csv("Qantum_2 - Eglin_Sept.csv ")

In [8]:
''' 
Filters:
        1. No Negative values for CBX
        2. Meter filtered for 10% of MAX for MIN and 90% of MAX for Clipping
        3. Irradiance above 200
        4. Start at 11 am, Peak hours/No Shading
Added Parameters: 
        5. DC Expected = daily_poa*string_count*IMP
        6. Heat Map: Summation(DC_Current)/DC_Expected
'''


for i in new_df.columns:
    if "Recombiner" in i:
        filtered_df1 = new_df[new_df.loc[:,i] > 1]
        
meter_min = 30000000 * 0.10
meter_max = 30000000 * 0.98


filtered_df3 = filtered_df1[(filtered_df1.loc[:, "Meter AC Power (W)"]>meter_min)&(filtered_df1.loc[:, "Meter AC Power (W)"]<meter_max)&(filtered_df1.loc[:,"Sensors - Irradiance reference (W/m²)"]>200)]

filtered_df3 = new_df.between_time("9:00", "17:00")
filtered_df3[filtered_df3 <= 0.00] = 0


C:\Users\dpinales\AppData\Local\Temp\ipykernel_101368\2493859364.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df3[filtered_df3 <= 0.00] = 0
C:\Users\dpinales\AppData\Local\Temp\ipykernel_101368\2493859364.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df3[filtered_df3 <= 0.00] = 0


In [ ]:

cbx = []
cbx_sum = []


for x in filtered_df3:
    if "Recombiner" in x:
        cbx.append(x)
        cbx_sum.append(filtered_df3[x].sum())
        
cbx_data = pd.DataFrame(cbx_sum, index=[cbx], columns=['Sum_of_CBX'])
cbx_data

In [ ]:
fig_dates = filtered_df3.index
fig_index = cbx_data.index
fig_heatdf = cbx_data.filter(["Estimated_Downstrings", "String_count"])
# fig_heatdf
fig = px.density_heatmap(fig_heatdf, x=fig_heatdf.index.values,y=fig_dates,)
fig.show()


In [ ]:
e_downstrings = sum(down_strings)
total_string = sum(string_count)
ratio_strings = e_downstrings/total_string

print(f"The estimated plants DC outage: {ratio_strings}")

In [ ]:
inverters = pd.read_csv("data\\Eglin - Nonclipping_8-25 INV.csv")
inverters = inverters.set_index([df_index])
inverters.index = pd.to_datetime(inverters.index)
inverters = inverters[~inverters.index.duplicated(keep='first')]
inverters = inverters.drop("Date (-05:00)", axis=1)
inverters = inverters.between_time("9:00", "17:00")

inv_sum = []
inv = []


for x in inverters.columns:
    if "INV" in x:
        inv.append(x)
        inv_sum.append(inverters[x].sum())
inv_df = pd.DataFrame(inv_sum, index=[inv], columns=['Sum_of_inv'])
inv_df

In [ ]:
'''Summation of CMB/INV'''


sum_of_cbx = cbx_data['Sum_of_CBX']
sum_of_inv = inv_df['Sum_of_inv']

def cmb_summation(a,b):
    total = (a/b)
    return total


cmb1 = []
blk1_sum = []
_summation1 = []

inv1 = []
inv1_blk = []

for inv_range in range(100, 108, 1):
    inv1.append(sum_of_inv.filter(like=str(inv_range), axis=0))

for i in range(1, len(inv1)):
    _inv1 = (sum(inv1[i]))
    inv1_blk.append(_inv1)    

for cmb_range in range(100, 108, 1):
    cmb1.append(sum_of_cbx.filter(like=str(cmb_range), axis=0))

for i in range(1,len(cmb1)):
    _cb1 = (sum(cmb1[i]))
    blk1_sum.append(_cb1)

for (v, n) in zip(blk1_sum, inv1_blk):
    _summation1.append(cmb_summation(v, n))
    

In [ ]:
inv2_dc = {
    "INV1",
    "INV2",
    "INV3",
    "INV4",
    "INV5",
    "INV6",
    "INV7",
}


cmb2 = []
blk2_sum = []

inv2 = []
inv2_blk = []

_summation2 = []

for inv_range in range(200, 208, 1):
    inv2.append(sum_of_inv.filter(like=str(inv_range), axis=0))

for i in range(1, len(inv2)):
    _inv2 = (sum(inv2[i]))
    inv2_blk.append(_inv2)    

for cmb_range in range(200, 208, 1):
    cmb2.append(sum_of_cbx.filter(like=str(cmb_range), axis=0))

for i in range(1,len(cmb2)):
    _cb2 = (sum(cmb2[i]))
    blk2_sum.append(_cb2)

for (v, n) in zip(blk2_sum, inv1_blk):
    _summation2.append(cmb_summation(v, n))


In [ ]:
total_summation = list(zip(_summation1, _summation2))

cbx_over_inv = pd.DataFrame(total_summation, index=[n for n in inv2_dc.keys()],columns=['BLK1 CBX100-107/INV DC', "BLK2 CBX200-207/INV DC"])
cbx_over_inv

In [ ]:
cbx_data.to_csv("Gulf I - 9-14.csv")